In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

import tensorflow as tf
import numpy as np
import math
import matplotlib.pyplot as plt
#%matplotlib inline

import cifar10_input

from discretization_utils import one_hot_to_thermometer
from discretization_utils import discretize_uniform
from discretization_attacks import adv_lspga

from cifar_model import Model
import cifar10_input

import torch
from wrapper import MyModel
import time
torch.set_printoptions(precision=20)

In [7]:
class blackbox:
    def __init__(self,model):
        self.model = model
        
    def attack_untargeted(self, x0, y0, alpha = 0.2, beta = 0.05, iterations = 1000):
        """ Attack the original image and return adversarial example
            model: (pytorch model)
            alpha: learning rate 
            beta: learning rate
            train_dataset: set of training data
            (x0, y0): original image
        """
        if self.model.predict(x0) != y0:
            print("Fail to classify the image. No need to attack.")
            return x0,0
    
        num_directions = 100
        best_theta, g_theta = None, float('inf')
        query_count = 0
        
        timestart = time.time()
        
        
        for i in range(num_directions):
            theta = torch.randn(x0.shape).type(torch.FloatTensor)
            initial_lbd = torch.norm(theta)
            theta = theta/torch.norm(theta)
            pred = self.model.predict(x0+np.array(initial_lbd*theta))
            if pred != y0:
                #print("new feasible initialization:", pred)
                query_count += 1
                lbd, count = self.fine_grained_binary_search( x0, y0, theta, initial_lbd, g_theta)
                query_count += count
                if lbd < g_theta:
                    best_theta, g_theta = theta,lbd
                    print("--------> Found distortion %.4f in iterationn %d" % (g_theta,i))
        timeend = time.time()
        print("==========> Found best distortion %.4f in %.4f seconds using %d queries" % (g_theta, timeend-timestart, query_count))
        
        
        g1 = 1.0
        theta, g2 = best_theta.clone(), g_theta
        torch.manual_seed(0)
        opt_count = 0
        stopping = 0.01
        prev_obj = 100000
        for i in range(iterations):
            print("iteration %d , distortion %.4f "% (i+1,g_theta))
            timestart = time.time()
            if g_theta < 2:
                print("============query number after distortion < 2 ==============:",opt_count+query_count)
                break
            
            gradient = torch.zeros(theta.size())
            #print("type of gradiennt", type(gradient))
            q = 10
            min_g1 = float('inf')
            for _ in range(q):
                u = torch.randn(theta.size()).type(torch.FloatTensor)
                u = u/torch.norm(u)
                #print("type of u", type(u))
                ttt = theta+beta * u
                ttt = ttt/torch.norm(ttt)
                g1, count = self.fine_grained_binary_search_local( x0, y0, ttt, initial_lbd = g2, tol=beta/500)
                opt_count += count
                #print("type of g1", type(g1))
                #print("type of g2", type(g2))
                
                gradient +=  (g1-g2)/beta * u
                #print("norm of gradient:", np.linalg.norm(gradient))
                if g1 < min_g1:
                    min_g1 = g1
                    min_ttt = ttt
            gradient = 1.0/q * gradient
    
            if (i+1)%50 == 0:
                print("Iteration %3d: g(theta + beta*u) = %.4f g(theta) = %.4f distortion %.4f num_queries %d" % (i+1, g1, g2, torch.norm(g2*theta), opt_count))
                if g2 > prev_obj-stopping:
                    break
                prev_obj = g2
    
            min_theta = theta
            min_g2 = g2
        
            for _ in range(5):
                #print("enter first for loop")
                new_theta = theta - alpha * gradient
                new_theta = new_theta/torch.norm(new_theta)
                new_g2, count = self.fine_grained_binary_search_local( x0, y0, new_theta, initial_lbd = min_g2, tol=beta/500)
                opt_count += count
                alpha = alpha * 2
                #print("alpha in the first for loop is: ",alpha)
                if new_g2 < min_g2:
                    min_theta = new_theta 
                    min_g2 = new_g2
                else:
                    break
            #print("alpha after first for loop :", alpha)
    
            if min_g2 >= g2:
                for _ in range(15):
                    #print("enter second for loop")
                    alpha = alpha * 0.25
                    new_theta = theta - alpha * gradient
                    new_theta = new_theta/torch.norm(new_theta)
                    new_g2, count = self.fine_grained_binary_search_local( x0, y0, new_theta, initial_lbd = min_g2, tol=beta/500)
                    opt_count += count
                    #print("alpha in the second for loop is: ",alpha)
                    if new_g2 < g2:
                        min_theta = new_theta 
                        min_g2 = new_g2
                        break
                #print("alpha after second for loop :", alpha)
    
            if min_g2 <= min_g1:
                theta, g2 = min_theta, min_g2
            else:
                theta, g2 = min_ttt, min_g1
    
            if g2 < g_theta:
                best_theta, g_theta = theta.clone(), g2
            
            #print(alpha)
#            print("%3d th iteration" % i)
            #print("current alpha:",alpha)
            if alpha < 1e-4:
                alpha = 1.0
                #print("Warning: not moving, g2 %lf gtheta %lf" % (g2, g_theta))
                beta = beta * 0.1
                if (beta < 1e-5):
                    break
            #print("time consuming in this iteration:",timeend - timestart)
            #print("iteration:",i)
            #print("query so far:", query_count+opt_count)
    
        #target = model.predict(x0 + g_theta*best_theta)
        #print("\nAdversarial Example Found Successfully: distortion %.4f target %d queries %d \nTime: %.4f seconds" % (g_theta, target, query_count + opt_count, timeend-timestart))
        print("thermometer")
        print("best distortion :", g_theta)
        print("number of queries :", opt_count+query_count)
        return np.array(g_theta*best_theta), opt_count+query_count
    def fine_grained_binary_search_fix(self,x0,y0,theta, initial_lbd = 1.0, tol=1e-5,current_best = float('inf'),num_query = 10):
        nquery = 0
        if initial_lbd > current_best: 
            if self.model.predict(x0+ np.array(current_best*theta)) == y0:
                nquery += 1
                return float('inf'), nquery
            lbd = current_best
        else:
            lbd = initial_lbd
    
        lbd_hi = lbd
        lbd_lo = 0.0
    
        while not np.isclose(lbd_hi,lbd_lo,tol):
            lbd_mid = (lbd_lo + lbd_hi)/2.0
            nquery += 1
            if self.model.predict(x0 + np.array(lbd_mid*theta)) != y0:
                lbd_hi = lbd_mid
            else:
                lbd_lo = lbd_mid
            if nquery > num_query:
                break
        comp_dec = (initial_lbd - lbd_hi)/initial_lbd
       # print("number of query before return for this direction:",nquery)
        return lbd_hi,comp_dec,nquery
    
    def fine_grained_binary_search_local(self, x0, y0, theta, initial_lbd = 1.0, tol=1e-5):
        nquery = 0
        lbd = initial_lbd
        #print("type of lbd before", type(lbd))
        lbd = np.array(lbd)
        #print("type of lbd after", type(lbd))
        
        if self.model.predict(x0+np.array(lbd*theta)) == y0:
            lbd_lo = lbd*1
            lbd_hi = lbd*1.01
            nquery += 1
            while self.model.predict(x0+np.array(lbd_hi*theta)) == y0:
                lbd_hi = lbd_hi*1.01
                nquery += 1
                if lbd_hi > 100:
                    return float('inf'), nquery
        else:
            lbd_hi = lbd*1
            lbd_lo = lbd*0.99
            nquery += 1
            while self.model.predict(x0+np.array(lbd_lo*theta)) != y0 :
                lbd_lo = lbd_lo*0.99
                nquery += 1
    
        #print("type of lbd_hi before while",type(lbd_hi))
       # print("type of lbd_lo before while",type(lbd_lo))
       # print("type of tol before while",type(tol))
        
        while (lbd_hi - lbd_lo) > tol:
            lbd_mid = (lbd_lo + lbd_hi)/2.0
            nquery += 1
            if self.model.predict(x0 + np.array(lbd_mid*theta)) != y0:
                lbd_hi = lbd_mid
            else:
                lbd_lo = lbd_mid
            #print("while loop in local binary",lbd_hi,lbd_lo,lbd_mid,lbd_hi - lbd_lo,tol)
        lbd_hi = np.array(lbd_hi)
        lbd_hi = torch.FloatTensor(lbd_hi)
        #print("size of lbd_hi",lbd_hi.size())
        #print("type of lbd_hi ", type(lbd_hi))

        return lbd_hi, nquery
    
    def fine_grained_binary_search(self, x0, y0, theta, initial_lbd, current_best):
        nquery = 0
        if initial_lbd > current_best: 
            if self.model.predict(x0+ np.array(current_best*theta)) == y0:
                nquery += 1
                return float('inf'), nquery
            lbd = current_best
        else:
            lbd = initial_lbd
        
        lbd_hi = lbd
        lbd_lo = 0.0
    
        while (lbd_hi - lbd_lo) > 1e-5:
            lbd_mid = (lbd_lo + lbd_hi)/2.0
            nquery += 1
            if self.model.predict(x0 + np.array(lbd_mid*theta)) != y0:
                lbd_hi = lbd_mid
            else:
                lbd_lo = lbd_mid
        return lbd_hi, nquery
    
    

In [8]:
levels = 16

sess = tf.Session(config=tf.ConfigProto(log_device_placement=False))
cifar = cifar10_input.CIFAR10Data()
model = Model('../models/thermometer_advtrain/',
              sess, tiny=False, mode='eval',
              thermometer=True, levels=levels)
model = MyModel(model,sess,levels,[0.0,255.0])

image = np.array(cifar.eval_data.xs[:100],dtype=np.float32)
labels = cifar.eval_data.ys[:100]

new_img = image / 255.0
print("type of image:", type(new_img))
count = []
for i in range(20):
    label = model.predict(new_img[i])
    if label == labels[i]:
        count.append(1)
    else:
        count.append(0)
    
print("accuracy of this model is:", sum(count)/len(count))

attack = blackbox(model)

Called
INFO:tensorflow:Restoring parameters from ../models/thermometer_advtrain/checkpoint-68000
restored
type of image: <class 'numpy.ndarray'>
accuracy of this model is: 0.95


In [ ]:
timestart = time.time()
label = model.predict(new_img[0])
timeend = time.time()
print("it costs %.4f seconds to query once, the label is %3d "%(timeend-timestart, label))

it costs 0.0106 seconds to query once, the label is   3 


In [ ]:
dist = []
count = []
for i in range(15):
    print("=========================== this is image ",i+1,"==================================")
    print("original label:",labels[i+1])
    mod,queries = attack.attack_untargeted(new_img[i+1],labels[i+1],alpha = 1, beta = 0.01, iterations = 1000)
    dist.append(np.linalg.norm(mod))
    count.append(queries)
    
    
index = np.nonzero(count)
index = list(index)[0].tolist()

avg_distortion = np.mean(np.array(dist)[index])
avg_count = np.mean(np.array(count)[index])
print("the average distortion for %2d images :"%(len(index)),avg_distortion)
print("the distortions for 15 images :")
for i in dist:
    print(i)
    
print("the number of queries for %2d images :"%(len(index)), avg_count)    
print("the number of queries for 15 images :")
for j in count:
    print(j)

=========================== this is image  1 ==================================
original label: 8
--------> Found distortion 19.5305 in iterationn 0
--------> Found distortion 13.2667 in iterationn 1
--------> Found distortion 11.2359 in iterationn 4
--------> Found distortion 10.8403 in iterationn 68
==========> Found best distortion 10.8403 in 3.2613 seconds using 282 queries
iteration 1 , distortion 10.8403 
iteration 2 , distortion 10.4176 
iteration 3 , distortion 10.4176 
iteration 4 , distortion 10.3422 
iteration 5 , distortion 10.3366 
iteration 6 , distortion 10.3346 
iteration 7 , distortion 10.3312 
iteration 8 , distortion 10.3301 
thermometer
best distortion : tensor(10.32923316955566406250)
number of queries : 3190
=========================== this is image  2 ==================================
original label: 8
--------> Found distortion 10.1889 in iterationn 0
--------> Found distortion 8.5416 in iterationn 1
--------> Found distortion 1.4195 in iterationn 2
--------> F

iteration 4 , distortion 1.7362 
============query number after distortion < 2 ==============: 981
thermometer
best distortion : tensor(1.73616111278533935547)
number of queries : 981
=========================== this is image  9 ==================================
original label: 1
Fail to classify the image. No need to attack.
=========================== this is image  10 ==================================
original label: 0
--------> Found distortion 7.5105 in iterationn 0
--------> Found distortion 7.3749 in iterationn 2
--------> Found distortion 6.2231 in iterationn 3
--------> Found distortion 6.0347 in iterationn 9
--------> Found distortion 5.8316 in iterationn 12
--------> Found distortion 4.9189 in iterationn 41
==========> Found best distortion 4.9189 in 3.6137 seconds using 317 queries
iteration 1 , distortion 4.9189 
iteration 2 , distortion 4.8300 
iteration 3 , distortion 4.6638 
iteration 4 , distortion 4.6196 
iteration 5 , distortion 4.5780 
iteration 6 , distortion 4.5